# Load Libraries

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

from autotrain.params import LLMTrainingParams
from autotrain.project import AutoTrainProject

HUGGINGFACE_BASE_MODEL = "meta-llama/Llama-3.2-3B"
DATA = "iamraymondlow/sharegpt_censored"  # iamraymondlow/sharegpt_censored, iamraymondlow/sharegpt_uncensored
PROJECT_NAME = "test-project"  # llama-3.2-3B-instructiontuned-censored
RANDOM_SEED = 42
BACKEND = "spaces-cpu-basic"


# Create Hugging Face Finetuning Project

In [ ]:
params = LLMTrainingParams(
    model=HUGGINGFACE_BASE_MODEL,
    project_name=PROJECT_NAME,
    data_path=DATA,
    train_split="train",
    model_max_length=128000,
    padding="right",
    trainer="sft",
    log="tensorboard",
    auto_find_batch_size=True,
    mixed_precision="none",
    lr=1e-5,
    epochs=1,  # 100
    batch_size=4,
    gradient_accumulation=4,
    optimizer="adamw_torch",
    scheduler="cosine",
    seed=RANDOM_SEED,
    chat_template="tokenizer",
    quantization="none",
    target_modules="all-linear",
    merge_adapter=True,
    peft=True,
    max_prompt_length=64000,
    max_completion_length=64000,
    text_column="text",
    push_to_hub=True,
    username=os.environ.get("HF_USERNAME"),
    token=os.environ.get("HF_TOKEN"),    
)

project = AutoTrainProject(params=params, backend=BACKEND, process=True)
project.create()

# Combine Hugging Face Adapter and Upload to Hugging Face

In [7]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from peft import PeftModel
from huggingface_hub import HfApi

pretrained_model_name = "google/gemma-2-27b"
adapter_name = "iamraymondlow/gemma-2-27b-instructiontuned-uncensored"
repo_name = "gemma-2-27b-instructiontuned-uncensored_merged"
username = "iamraymondlow"

# Load the pre-trained base model
base_model = AutoModelForSequenceClassification.from_pretrained(pretrained_model_name)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)

# Load the adapter using PeftModel (replace with the correct adapter path)
peft_model = PeftModel.from_pretrained(base_model, adapter_name)

# Merge the adapter into the pre-trained model
peft_model.merge_and_unload()

# Save the merged model locally
merged_model_path = "merged-model"
peft_model.save_pretrained(merged_model_path)
tokenizer.save_pretrained(merged_model_path)

# Initialize Hugging Face API and create a new repository
api = HfApi()
api.create_repo(repo_id=f"{username}/{repo_name}", exist_ok=True)

# Upload the merged model to the Hugging Face Hub
peft_model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)


config.json:   0%|          | 0.00/892 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/42.8k [00:00<?, ?B/s]

model-00001-of-00024.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]